In [1]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize

#%matplotlib qt
from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
import PlotUtil
from notebooks.optimization_problems.constraints import Requirements
%load_ext autoreload

%autoreload 2

In [2]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "example_data.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [3]:
N_passes = 20

instances_df = instances_500km_df.loc[0:N_passes,:]

In [4]:
from notebooks.optimization_problems.pointing_problem import PointingProblem
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 60.)

modcods_df = pd.read_pickle('dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df['isend'].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
requirements.min_throughput = 0.5e9
#requirements.max_throughput = 5e9

problem = PointingProblem(instances_df, sys_param, requirements=requirements)

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

converge_algo_settings = {'pop_size': 200, 'n_offsprings': 100}
unconverge_algo_settings = {'pop_size': 100, 'n_offsprings': 20}

converge_term_settings = {'f_tol': 0.001, 'n_max_gen': 2000, 'n_max_evals': 200000}
unconverge_term_settings =  {'f_tol': 0.005, 'n_max_gen': 100, 'n_max_evals': 2000}

converge = True
if converge:
    algo_settings = converge_algo_settings
    term_settings = converge_term_settings
    postfix = '_converged'
else:
    algo_settings = unconverge_algo_settings
    term_settings = unconverge_term_settings
    postfix = '_unconverged'

from pymoo.util.termination.default import MultiObjectiveDefaultTermination

termination = MultiObjectiveDefaultTermination(
    #x_tol=1e-8,
    #cv_tol=1e-6,
    f_tol=term_settings['f_tol'], #f_tol=0.005,
    nth_gen=100,
    n_last=30,
    n_max_gen=term_settings['n_max_gen'], #n_max_gen=1000,
    n_max_evals=term_settings['n_max_evals'], #n_max_evals=100000
)

termination = MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)

from pymoo.algorithms.nsga2 import NSGA2

algorithm = NSGA2(
    pop_size=algo_settings['pop_size'],
    n_offsprings=algo_settings['n_offsprings'],
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True,
)

res = minimize(problem,
               algorithm,
               termination=termination,
               seed=1,
               #save_history=True,
               verbose=True
               )

#res = minimize(problem, GA())
# pool.close()
# problem.pool.close()

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type array(pyobject, 1d, C)[0m
[0m[1mDuring: typing of argument at C:\git\hermes-optimization\models\vcm.py (32)[0m
[1m
File "..\models\vcm.py", line 32:[0m
[1mdef compute_passes_throughput(tof_s_list, fspl_dB_list, Ptx_dBm_array, Gtx_dBi, GT_dBK, B_Hz,
    <source elided>
                              alpha, EsN0_req_dB_array, eta_bitsym_array, min_margin_dB):
[1m    linktime_s_array = np.empty(len(tof_s_list))
[0m    [1m^[0m[0m


In [ ]:
import pickle

pickle.dump(res, open('pointing_res.pkl', 'wb'))

In [ ]:
res = pickle.load(open('pointing_res.pkl', 'rb'))

In [ ]:
x_pass = res.X[:, problem.x_indices['pass']].astype('bool')
x_Ptx_dBm = res.X[:, problem.x_indices['power']].astype('float64')
x_Ptx_dBm[~x_pass] = np.NaN
x_Ptx_dBm = np.nanmax(x_Ptx_dBm, axis=1)
#x_modcod = np.squeeze(res.X[:, problem.x_indices['modcod']].astype('int64'))

f_energy = res.F[:,1] / 1e3 # Kilo Joule
f_throughput = ((res.F[:,0] * -1) / 1e9)   # Gigabit

In [ ]:
import colorcet as cc

from matplotlib.lines import Line2D
import numpy as np
import matplotlib.pyplot as plt
#from PlotUtil import apply_report_formatting, apply_report_formatting_single
import matplotlib.colors as mcol
import matplotlib.cm as cm

isend_req = 10.69

fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

cm1 = mcol.LinearSegmentedColormap.from_list("BlueRed",["c","r"])
#cm1 = mcol.LinearSegmentedColormap.from_list("BlueRed",["#4a569d","#dc2424"])
#cm1 = cc.cm.linear_kryw_5_100_c67_r
#cm1 = cc.cm.CET_L18

min_Ptx = np.nanmin(x_Ptx_dBm)
max_Ptx = np.nanmax(x_Ptx_dBm)

cnorm = mcol.Normalize(vmin=min_Ptx,vmax=max_Ptx)

cpick = cm.ScalarMappable(norm=cnorm,cmap=cm1)
cpick.set_array([])

cp = cpick.to_rgba(x_Ptx_dBm.tolist())
ax.scatter(f_energy, f_throughput, c=cp, marker='.', s=1)

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Throughput [Gbit]")
cb = plt.colorbar(cpick,label="Maximum power [dBm]")
cb.ax.tick_params(labelsize=8)
cb.ax.yaxis.label.set_size(8)

PlotUtil.apply_report_formatting(4.72, 2.76)
plt.tight_layout()

#fig.savefig('D:/git/thesis_report_ae/figures/mc_pareto%s.pdf' % postfix)
plt.show()
plt.close()

In [ ]:
x_bandwidth = res.X[:, problem.x_indices['bandwidth']].astype('int')
B_Hz = sys_param.B_Hz_array[np.squeeze(x_bandwidth)] / 1e6

fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

ax.scatter(B_Hz, f_throughput, marker='.', s=2)
#ax.set_xscale('log')
ax.set_xlabel("Bandwidth [MHz]")
ax.set_ylabel("Throughput [Gbit]")
#ax.set_xlim((, 11))
ax.set_ylim((0, 1200))

PlotUtil.apply_report_formatting(3.146, 2.76)
plt.tight_layout()

#fig.savefig('D:/git/thesis_report_ae/figures/vcm_modcod_vs_throughput.pdf')

In [ ]:
fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

ax.scatter(np.sum(x_pass, axis=1), f_throughput, marker='.', s=1)
ax.set_xlabel("Number of passes used")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim((0, 20))
ax.set_ylim((0, 1200))

PlotUtil.apply_report_formatting(3.146, 2.76)
plt.tight_layout()

#fig.savefig('D:/git/thesis_report_ae/figures/vcm_modcod_vs_throughput.pdf')

In [ ]:
fig = plt.figure(figsize=(3.2, 2.4))

import matplotlib.pyplot as plt
from pymoo.performance_indicator.hv import Hypervolume

# create the performance indicator object with reference point (4,4)
metric = Hypervolume(ref_point=np.array([0,1e18]))

# collect the population in each generation
pop_each_gen = [a.pop for a in res.history]

# receive the population in each generation
obj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]

# calculate for each generation the HV metric
hv = [metric.calc(f) for f in obj_and_feasible_each_gen]

# function evaluations at each snapshot
n_evals = np.array([a.evaluator.n_eval for a in res.history])

# visualze the convergence curve
plt.plot(n_evals, hv, '-o', markersize=3)
plt.title("Convergence")
plt.xlabel("Function Evaluations")
plt.ylabel("Hypervolume")
PlotUtil.apply_report_formatting()
plt.tight_layout()
plt.grid()
#plt.savefig('D:/git/thesis_report_ae/figures/link_budget_hypervolume%s.svg' % postfix)
fig.savefig('D:/git/thesis_report_ae/figures/visibility_hypervolume%s.pdf' % postfix)
plt.show()
plt.close()
